# Connection Improvements

The class **`nengolib.Connection()`** is a replacement for **`nengo.Connection()`**. This new class adds a bias activity to the pre-synaptic population. This is equivalent to adding a neuron with a flat tuning curve across the entire range of represented values. This is also equivalent to a change in the biases of the post-synaptic population.

This improves the decoding of any function on average, provided the regularization is set correctly on the decoder solver. The improvement is most significant for small numbers of neurons (as demonstrated below) and for constant functions. 

In [ ]:
%pylab inline
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

import numpy as np

import nengo
from nengo.utils.numpy import rmse

import nengolib

In [ ]:
solver = nengo.solvers.LstsqL2(reg=0.1)

trials = []
for _ in range(50):

    with nengolib.Network() as model:
        stim = nengo.Node(output=lambda t: np.sin(5*t*2*np.pi))
        x = nengo.Ensemble(5, 1, neuron_type=nengo.LIFRate())
        default = nengo.Node(size_in=1)
        improved = nengo.Node(size_in=1)

        nengo.Connection(stim, x, synapse=None)
        nengo.Connection(x, default, solver=solver)
        nengolib.Connection(x, improved, solver=solver)

        p_default = nengo.Probe(default)
        p_improved = nengo.Probe(improved)
        p_stim = nengo.Probe(stim, synapse=0.005)

    with nengo.Simulator(model) as sim:
        sim.run(0.5, progress_bar=False)

    trials.append(rmse(
        sim.data[p_default], sim.data[p_stim]) - rmse(sim.data[p_improved], sim.data[p_stim]))

pylab.figure()
pylab.title("Improvement")
pylab.hist(np.asarray(trials))
pylab.xlabel("Decrease in RMSE")
pylab.ylabel("Density")
pylab.show()